In [1]:
from selenium import webdriver
import time
import pandas as pd
import operator

In [2]:
chrome_path = r"C:\Users\saxou\Desktop\PYTHON HACK\Web Scrapping\chromedriver_win32 (1)\chromedriver.exe"
driver = webdriver.Chrome(chrome_path)
file2 = open('models.txt','r')
model_names = set([x.replace('\n','') for x in file2.readlines()])
brands_text = set(['google','huawei','nokia','samsung','ulefone','meizu','apple','wileyfox','wiko','motorola','xiaomi','cat','crosscall','doro','alcatel','asus','fairphone','land-rover','ruggear','gigaset','honor','lg','oneplus','oppo','ruggear','sony','renewd'])
colors_text = set(['space grey','prism','green','gold','purple','ceramic black','prism white','space gray','goud','rose gold','midnight green','keramisch','silver','wit','zilver','rose gold','grijs','rood','black','yellow','white','red','magenta','cyan','zwart','groen','blauw','paars','rosé goud','geel','oranje','brons','bruin','roze'])

In [3]:
driver.get("https://onlyhere.nl/product-category/smartphones/")
time.sleep(1)
SCROLL_PAUSE_TIME = 0.5

# Get scroll height
last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    # Scroll down to bottom
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait to load page
    time.sleep(SCROLL_PAUSE_TIME)

    # Calculate new scroll height and compare with last scroll height
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

In [4]:
hrefs=[]
href_1 = driver.find_element_by_class_name("products")
href_2 = href_1.find_elements_by_class_name("main-container")
#print(len(href_2))
hrefcss =[]
for h in href_2:
    hrefcss.append(h.find_element_by_css_selector('a'))
    
hrefs = []
for h in hrefcss:
    hrefs.append(h.get_attribute("href"))


#print(hrefs)

In [5]:
all_df = pd.DataFrame(columns=['Brand','Price','Storage',"Link","Color","Model","Competitor"])

In [8]:
for href in hrefs:
    tmp={}
    driver.get(href)
    time.sleep(1)
    
    tmp['Link']=href

    pricel_class = driver.find_element_by_class_name("price")
    pricel = pricel_class.find_elements_by_class_name("amount")
    if len(pricel) >= 2:
        tmp['Price']=pricel[1].text
    else:
        tmp['Price']=pricel[0].text

    title = driver.find_element_by_class_name("product_title")
    title_arr = title.text.split(' ')
    print(title_arr)
    
    if title_arr[0].lower() == "iphone":
        label = "Apple"
        tmp['Brand']=label
    else:
        label = title_arr[0]
        tmp['Brand']=label

    for elem in title_arr:
        if 'GB' in elem:
            tmp['Storage']=elem
    
    name=title.text.lower()
    print(name)

    try:
        colorels = driver.find_elements_by_class_name("attached")
        Cs = []
        for c in colorels:
            Cs.append(c.text)
        color = ' / '.join(Cs)
        tmp['Color']=color

    except:
        tmp2 = {}
        for clr in colors_txt:
            #print(clr)
            if clr in name:
                tmp2[clr] = len(clr)
        tmp['Color']=max(tmp2.items(), key=operator.itemgetter(1))[0]
        #print(max(tmp.items(), key=operator.itemgetter(1))[0])


    tmp1 = {}
    for mdl in model_names:
        #print(mdl)
        if mdl.lower() in name:
            tmp1[mdl] = len(mdl)
    tmp['Model']=max(tmp1.items(), key=operator.itemgetter(1))[0]
    #print(max(tmp.items(), key=operator.itemgetter(1))[0])
      
    tmp['Competitor']="OnlyHere"    
    all_df = all_df.append(tmp,ignore_index=True)
    print(tmp)

['Apple', 'iPhone', '11', '64GB']
apple iphone 11 64gb
{'Link': 'https://onlyhere.nl/product/apple-iphone-11-64gb/', 'Price': '€715.00', 'Brand': 'Apple', 'Storage': '64GB', 'Color': 'Black', 'Model': 'Iphone 11', 'Competitor': 'OnlyHere'}
['iPhone', '11', 'Pro', 'Max', '512GB']
iphone 11 pro max 512gb
{'Link': 'https://onlyhere.nl/product/11-pro-max-512gb/', 'Price': '€1,375.00', 'Brand': 'Apple', 'Storage': '512GB', 'Color': 'Gold / Silver', 'Model': 'Iphone 11 Pro Max', 'Competitor': 'OnlyHere'}
['MSI', 'INFINITE', 'S', '8RA-032EU', 'Black']
msi infinite s 8ra-032eu black
{'Link': 'https://onlyhere.nl/product/msi-infinite-s-8ra-032eu-black/', 'Price': '€600.00', 'Brand': 'MSI', 'Color': '', 'Model': '3', 'Competitor': 'OnlyHere'}
['Nokia', '3.1', '16GB', 'Blauw']
nokia 3.1 16gb blauw
{'Link': 'https://onlyhere.nl/product/nokia-3-1-16gb-blauw/', 'Price': '€110.00', 'Brand': 'Nokia', 'Storage': '16GB', 'Color': '', 'Model': '3.1', 'Competitor': 'OnlyHere'}
['Nokia', '5', '16GB', 'Blau

In [9]:
len(hrefs)

14

In [10]:
all_df.to_excel('onlyhere.xls')
all_df

,Brand,Price,Storage,Link,Color,Model,Competitor
0,Apple,€715.00,64GB,https://onlyhere.nl/product/apple-iphone-11-64gb/,Black,Iphone 11,OnlyHere
1,Apple,"€1,375.00",512GB,https://onlyhere.nl/product/11-pro-max-512gb/,Gold / Silver,Iphone 11 Pro Max,OnlyHere
2,MSI,€600.00,NaN,https://onlyhere.nl/product/msi-infinite-s-8ra...,,3,OnlyHere
3,Nokia,€110.00,16GB,https://onlyhere.nl/product/nokia-3-1-16gb-blauw/,,3.1,OnlyHere
4,Nokia,€100.00,16GB,https://onlyhere.nl/product/nokia-5-1-16gb-blauw/,,5,OnlyHere
5,Nokia,€190.00,32GB,https://onlyhere.nl/product/nokia-7-1-32gb-blauw/,,7.1,OnlyHere
6,Apple,€635.00,128GB,https://onlyhere.nl/product/apple-iphone-xr-12...,Blue,Iphone Xr,OnlyHere
7,Apple,€595.00,64GB,https://onlyhere.nl/product/apple-iphone-xr-64gb/,Black,Iphone Xr,OnlyHere
8,Apple,€740.00,64GB,https://onlyhere.nl/product/apple-iphone-xs-64gb/,Space Gray,Iphone Xs,OnlyHere
9,Apple,€670.00,64GB,https://onlyhere.nl/product/iphone-xs-64gb-zgan/,,Iphone Xs,OnlyHere
